In [1]:
from PIL import Image
import torch as th
import numpy as np
from UNet import UNet
from data_prep.preprocess import Preprocess


# Load data

In [ ]:
#TODO load data

input_height = #TODO
input_width = #TODO
input_features = #TODO

# Define the model

In [ ]:
model = UNet(INPUT_HEIGHT = input_height, INPUT_WIDTH = input_width, INPUT_FEATURE_NUMBER = input_features)

criterion = th.nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model.parameters(), lr = 0.01)

# Train

In [ ]:
epochs = 100

data = #TODO

for epoch in range(epochs):
    batch = #TODO
    target = #TODO

    output = model(batch)

    loss = criterion(output, target)

    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()